In [3]:
from qiskit import *
from qiskit.circuit.classical import expr

In [ ]:
def prepare_CCZ() -> QuantumCircuit:
    """ Prepare a 3 qubits CCZ state.
    """
    reg = QuantumRegister(3)
    qc = QuantumCircuit(reg)
    qc.h(reg)
    qc.ccz(reg[0], reg[1], reg[2])
    return qc

In [ ]:
def prepare_i_state(num: int) -> QuantumCircuit:
    """ Prepare the |i> = (|0> + i|1>)/sqrt(2) state.
    """
    qc = QuantumCircuit(num)
    for i in range(num):
        qc.h(i)
        qc.s(i)
    return qc

In [ ]:
def prepare_semiauto_magic_state() -> QuantumCircuit:
    reg_ccz = QuantumRegister(3, 'ccz')
    reg_i = QuantumRegister(3, 'i')
    reg_anc = QuantumRegister(1, 'ancilla')
    qc = QuantumCircuit(reg_ccz, reg_i, reg_anc)

    #z parity check then cx
    qc.cx(reg_i[0], reg_anc[0])
    qc.cx(reg_anc[0], reg_i[0])
    qc.cx(reg_i[0], reg_anc[0])

    #z parity check then cx
    qc.ccx(reg_ccz[0], reg_ccz[2], reg_anc[0])
    qc.cx(reg_anc[0], reg_i[1])
    qc.ccx(reg_ccz[0], reg_ccz[2], reg_anc[0])

    #z parity check then cx
    qc.ccx(reg_ccz[0], reg_ccz[1], reg_anc[0])
    qc.cx(reg_anc[0], reg_i[2])
    qc.ccx(reg_ccz[0], reg_ccz[1], reg_anc[0])

    qc.h(reg_ccz[0])
    return qc

In [ ]:
def z_parity_check(num: int) -> QuantumCircuit:
    """
    Sets anc qubit to the Z parity of the data qubits.
    """
    reg_data = QuantumRegister(num, 'data')
    reg_anc = QuantumRegister(1, 'ancilla')
    qc = QuantumCircuit(reg_data, reg_anc)

    for i in range(num):
        qc.cx(reg_data[i], reg_anc[0])
    return qc

In [ ]:
def x_parity_check(num: int) -> QuantumCircuit:
    """
    Sets anc qubit to the X parity of the data qubits.
    """
    reg_data = QuantumRegister(num, 'data')
    reg_anc = QuantumRegister(1, 'ancilla')
    qc = QuantumCircuit(reg_data, reg_anc)

    for i in range(num):
        qc.h(reg_data[i])
        qc.cx(reg_data[i], reg_anc[0])
        qc.h(reg_data[i])
    return qc

In [ ]:
def interact() -> QuantumCircuit:
    reg_inputs = QuantumRegister(5, 'input')
    reg_ccz = QuantumRegister(3, 'ccz')
    reg_anc = QuantumRegister(1, 'ancilla')
    reg_c = ClassicalRegister(2, 'classical')

    qc = QuantumCircuit(reg_inputs, reg_ccz, reg_anc, reg_c)

    qc.append(z_parity_check(5), reg_inputs[:] + [reg_anc[0]])
    qc.cx(reg_anc[0], reg_ccz[0])
    qc.append(z_parity_check(5).inverse(), reg_inputs[:] + [reg_anc[0]])

    qc.append(z_parity_check(2), reg_inputs[1:3] + [reg_anc[0]])
    qc.cx(reg_anc[0], reg_ccz[1])
    qc.append(z_parity_check(2).inverse(), reg_inputs[1:3] + [reg_anc[0]])

    qc.append(z_parity_check(2), reg_inputs[0:1] + reg_inputs[2:3] + [reg_anc[0]])
    qc.cx(reg_anc[0], reg_ccz[2])
    qc.append(z_parity_check(2).inverse(), reg_inputs[0:1] + reg_inputs[2:3] + [reg_anc[0]])

    qc.measure(reg_ccz[1:3], reg_c)

    return qc

In [ ]:
from qiskit.circuit.classical import expr

def reaction_limited_basis_choice() -> QuantumCircuit:
    reg_c = ClassicalRegister(2, 'ccz_post_measure')
    reg_i = QuantumRegister(3, 'i')
    reg_ci = ClassicalRegister(3, 'i_post_measure')
    qc = QuantumCircuit(reg_c, reg_i, reg_ci)

    condition = expr.bit_xor(reg_c[0], reg_c[1])
    with qc.if_test(condition):
        qc.h(reg_i[0])
    condition = reg_c[0]
    with qc.if_test(condition):
        qc.h(reg_i[1])
    condition = reg_c[1]
    with qc.if_test(condition):
        qc.h(reg_i[2])
    
    qc.h(reg_i)
    qc.measure(reg_i, reg_ci)

    return qc

In [ ]:
from qiskit.circuit.classical import expr

def backdated_X_frame() -> QuantumCircuit:
    reg_ccz = QuantumRegister(1, 'ccz')
    reg_ccz_c = ClassicalRegister(2, 'ccz_post_measure')
    reg_i = ClassicalRegister(3, 'i_post_measure')
    qc = QuantumCircuit(reg_ccz, reg_ccz_c, reg_i)

    condition = expr.bit_xor(reg_ccz_c[0], expr.bit_xor(reg_ccz_c[1], expr.bit_xor(reg_i[0], expr.bit_xor(reg_i[1], reg_i[2]))))
    with qc.if_test(condition):
        qc.x(reg_ccz[0])
        
    return qc

In [ ]:
def start_uncompute_b() -> QuantumCircuit:
    reg_in = QuantumRegister(3, 'input') # reg in is the bits (a[k-1], b[k-1], (a+b)[k-1])
    reg_b = QuantumRegister(1, 'b') # reg b is the b[k] bit
    reg_b_c = ClassicalRegister(1, 'b_post_measure')
    reg_i = QuantumRegister(3, 'i')
    reg_anc = QuantumRegister(1, 'ancilla') # used as target for parity checks
    qc = QuantumCircuit(reg_in, reg_b, reg_b_c, reg_i, reg_anc)

    qc.h(reg_b[0])
    qc.measure(reg_b[0], reg_b_c[0])

    qc.append(z_parity_check(2), [reg_in[0], reg_in[1], reg_anc[0]])
    qc.cx(reg_anc[0], reg_i[0])
    qc.append(z_parity_check(2).inverse(), [reg_in[0], reg_in[1], reg_anc[0]])

    qc.append(z_parity_check(2), [reg_in[0], reg_in[2], reg_anc[0]])
    qc.cx(reg_anc[0], reg_i[1])
    qc.append(z_parity_check(2).inverse(), [reg_in[0], reg_in[2], reg_anc[0]])
    
    qc.append(z_parity_check(2), [reg_in[1], reg_in[2], reg_anc[0]])
    qc.cx(reg_anc[0], reg_i[2])
    qc.append(z_parity_check(2).inverse(), [reg_in[1], reg_in[2], reg_anc[0]])

    return qc

In [ ]:
def reaction_limited_basis_choice_uncompute() -> QuantumCircuit:
    reg_b_c = ClassicalRegister(1, 'b_post_measure')
    reg_ccz_c = ClassicalRegister(2, 'ccz_post_measure')
    reg_i = QuantumRegister(3, 'i')
    reg_i_c = ClassicalRegister(3, 'i_post_measure')
    qc = QuantumCircuit(reg_b_c, reg_ccz_c, reg_i, reg_i_c)

    condition = reg_b_c[0]
    with qc.if_test(condition):
        qc.h(reg_i[0])

    condition = expr.bit_xor(reg_b_c[0], reg_ccz_c[0])
    with qc.if_test(condition):
        qc.h(reg_i[1])

    condition = expr.bit_xor(reg_b_c[0], reg_ccz_c[1])
    with qc.if_test(condition):
        qc.h(reg_i[2])  
    
    qc.h(reg_i)
    qc.measure(reg_i, reg_i_c)
    
    return qc

In [ ]:
def backdated_Z_frame() -> QuantumCircuit:
    reg_in = QuantumRegister(4, 'input') # reg in is the bits (a[k-1], b[k-1], (a+b)[k-1], a[k])
    reg_b_c = ClassicalRegister(1, 'b_post_measure')
    reg_ccz = QuantumRegister(1, 'ccz')
    reg_ccz_c = ClassicalRegister(2, 'ccz_post_measure')
    reg_i_c_1 = ClassicalRegister(3, 'i_post_measure_1')
    reg_i_c_2 = ClassicalRegister(3, 'i_post_measure_2')
    qc = QuantumCircuit(reg_in, reg_b_c, reg_ccz, reg_ccz_c, reg_i_c_1, reg_i_c_2)

    # ccz[0] xor i1[1] xor i2[0] xor i2[1]
    condition = expr.bit_xor(reg_ccz_c[0], expr.bit_xor(reg_i_c_1[1], expr.bit_xor(reg_i_c_2[0], reg_i_c_2[1])))
    with qc.if_test(condition):
        qc.z(reg_in[0])

    # ccz[1] xor i1[2] xor i2[0] xor i2[2]
    condition = expr.bit_xor(reg_ccz_c[1], expr.bit_xor(reg_i_c_1[2], expr.bit_xor(reg_i_c_2[0], reg_i_c_2[2])))
    with qc.if_test(condition):
        qc.z(reg_in[1])

    # b_c[0] xor ccz[0] xor ccz[1] xor i1[1] xor i1[2] xor i2[1] xor i2[2]
    condition = expr.bit_xor(reg_b_c[0], expr.bit_xor(reg_ccz_c[0], expr.bit_xor(reg_ccz_c[1], expr.bit_xor(reg_i_c_1[1], expr.bit_xor(reg_i_c_1[2], expr.bit_xor(reg_i_c_2[1], reg_i_c_2[2]))))))
    with qc.if_test(condition):
        qc.z(reg_in[2])
    
    condition = reg_b_c[0]
    with qc.if_test(condition):
        qc.z(reg_in[3])
        qc.z(reg_ccz[0])

    condition = expr.bit_and(reg_ccz_c[0], reg_ccz_c[1])
    with qc.if_test(condition):
        qc.z(reg_in[0])
        qc.z(reg_in[1])

    condition = expr.bit_and(reg_b_c[0], reg_ccz_c[0])
    with qc.if_test(condition):
        qc.z(reg_in[0])
        qc.z(reg_in[2])

    condition = expr.bit_and(reg_b_c[0], reg_ccz_c[1])
    with qc.if_test(condition):
        qc.z(reg_in[1])
        qc.z(reg_in[2])

    return qc

In [ ]:
def compute_addition() -> QuantumCircuit:
    reg_in = QuantumRegister(5, 'input') # reg in is the bits (a[k-1], b[k-1], (a+b)[k-1], a[k], b[k])
    reg_ccz = QuantumRegister(3, 'ccz')
    reg_ccz_c = ClassicalRegister(2, 'ccz_post_measure')
    reg_i = QuantumRegister(3, 'i')
    reg_i_c = ClassicalRegister(3, 'i_post_measure')
    reg_anc = QuantumRegister(1, 'ancilla') # used as target for parity checks
    qc = QuantumCircuit(reg_in, reg_ccz, reg_ccz_c, reg_i, reg_i_c, reg_anc)

    qc.append(prepare_CCZ(), reg_ccz[:])
    qc.append(prepare_i_state(3), reg_i[:])

    qc.append(prepare_semiauto_magic_state(), reg_ccz[:] + reg_i[:] + [reg_anc[0]])

    qc.append(interact(), reg_in[:] + reg_ccz[:] + [reg_anc[0]] + reg_ccz_c[:])

    qc.append(reaction_limited_basis_choice(), reg_ccz_c[:] + reg_i[:] + reg_i_c[:])

    qc.append(backdated_X_frame(), reg_ccz[0:1] + reg_ccz_c[:] + reg_i_c[:])

    return qc

In [4]:
def uncompute_b() -> QuantumCircuit:
    reg_in = QuantumRegister(5, 'input') # reg in is the bits (a[k-1], b[k-1], (a+b)[k-1], a[k], b[k])
    reg_b_c = ClassicalRegister(1, 'b_post_measure')
    reg_ccz = QuantumRegister(1, 'ccz')
    reg_ccz_c = ClassicalRegister(2, 'ccz_post_measure')
    reg_i_c = ClassicalRegister(3, 'i_post_measure')
    reg_i_unc = QuantumRegister(3, 'i_uncompute')
    reg_i_unc_c = ClassicalRegister(3, 'i_uncompute_post_measure')
    reg_anc = QuantumRegister(1, 'ancilla') # used as target for parity checks
    qc = QuantumCircuit(reg_in, reg_b_c, reg_ccz, reg_ccz_c, reg_i_c, reg_i_unc, reg_i_unc_c, reg_anc)

    qc.append(prepare_i_state(3), reg_i_unc[:])

    qc.append(start_uncompute_b(), reg_in[0:3] + [reg_in[4]] + reg_b_c[:] + reg_i_unc[:] + [reg_anc[0]])

    qc.append(reaction_limited_basis_choice_uncompute(), reg_b_c[:] + reg_ccz_c[:] + reg_i_unc[:] + reg_i_unc_c[:])

    qc.append(backdated_Z_frame(), reg_in[0:4] + reg_b_c[:] + reg_ccz[:] + reg_ccz_c[:] + reg_i_c[:] + reg_i_unc_c[:])

    return qc